# 调用必备库

In [1]:
import time
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
from torch.autograd import Variable
import torch.nn.functional as F
import torch.optim as optim

np.random.seed(42)
torch.manual_seed(42)

# 先随便搭建个简单的神经网络


尝试求解一下两端点固定的自由振动问题

$u_{tt}-a^2u_{xx}=0$

$u(0,x)=sinx$

$u_t(0,x)=cosx$

$u(t,0)=0,u(l,t)=0$




## 神经网络求解

In [4]:
class zsrDGM_net(nn.Module):
    def __init__(self,numl,numn):
        # numl是有多少层隐藏层
        # numn是每层的神经元数量
        super(zsrDGM_net, self).__init__()
        self.input_layer = nn.Linear(2, numn)#前面的数字代表几个输入
        self.hidden_layers = nn.ModuleList([nn.Linear(numn, numn) for i in range(numl)])
        self.output_layer = nn.Linear(numn, 1)
    def forward(self, x):
        o = self.act(self.input_layer(x))
        for i, li in enumerate(self.hidden_layers):
            o = self.act(li(o))        
        out = self.output_layer(o)        
        return out
    def act(self, x):
        return x * torch.tanh(x)

In [5]:
class PDE():
    def __init__(self, net, a,l,t):
        self.net=net
        self.a=a
        self.l=l
        self.t=t
    def sample(self,size=2**8):        
        x = torch.cat((torch.rand([size, 1]) * self.t, torch.rand([size, 1]) * self.l), dim=1)
        x_init = torch.rand([size, 1]) * self.l
        x_initial = torch.cat((torch.zeros(size, 1), x_init), dim=1)
        x_boundary_left = torch.cat((torch.rand([size, 1])*self.t, torch.full([size, 1], 0)), dim=1)
        x_boundary_right = torch.cat((torch.rand([size, 1])*self.t, torch.full([size, 1], l)), dim=1)
        return x, x_initial, x_init, x_boundary_left, x_boundary_right
    def loss_func(self,size=2**8):
        x_train, x_initial, x_init, x_boundary_left, x_boundary_right = self.sample(size=size)
        x = Variable(x_train, requires_grad=True)
        d = torch.autograd.grad(net(x), x, grad_outputs=torch.ones_like(net(x)), create_graph=True)
        dt = d[0][:, 0].unsqueeze(-1)
        dx = d[0][:, 1].unsqueeze(-1)
        dxx = torch.autograd.grad(dx, x, grad_outputs=torch.ones_like(dx), create_graph=True)[0][:, 1].unsqueeze(-1)
        dtt = torch.autograd.grad(dt, x, grad_outputs=torch.ones_like(dt), create_graph=True)[0][:, 1].unsqueeze(-1)
        x2=Variable(x_initial, requires_grad=True)
        d1 = torch.autograd.grad(net(x2), x2, grad_outputs=torch.ones_like(net(x2)), create_graph=True)
        dt1 = d1[0][:, 0].unsqueeze(-1)
        loss_fn = nn.MSELoss(reduction='mean')
        loss1 = loss_fn(dtt, self.a**2*dxx)
        loss2 = loss_fn(net(x_initial), torch.sin(x_init))
        loss3 = loss_fn(net(x_boundary_left), torch.zeros([size, 1]))
        loss4 = loss_fn(net(x_boundary_right), torch.zeros([size, 1]))
        loss5 = loss_fn(dt1,1*torch.cos(x_init))
        loss = loss1 + loss2 + loss3 + loss4+ loss5
        return loss,loss1,loss2,loss3,loss4,loss5
        
        

In [6]:
class Train():
    def __init__(self, net, eq, BATCH_SIZE):
        self.errors = []
        self.BATCH_SIZE = BATCH_SIZE
        self.net = net
        self.model = eq
    def train(self, epoch, lr):
        optimizer = optim.Adam(self.net.parameters(), lr)
        avg_loss = 0
        for e in range(epoch):
            optimizer.zero_grad()
            loss,loss1,loss2,loss3,loss4,loss5 = self.model.loss_func(self.BATCH_SIZE)
            avg_loss = avg_loss + float(loss.item())
            loss.backward()
            optimizer.step()
            if e % 50 == 49:
                loss = avg_loss/50
                print("Epoch {} - lr {} -  loss: {}".format(e, lr, loss))
                avg_loss = 0

                error,loss1,loss2,loss3,loss4,loss5 = self.model.loss_func(2**12)
                self.errors.append(error.detach())
    def get_errors(self):
        return self.errors

In [9]:
net = zsrDGM_net(numl=7, numn=300)
a=1
l=np.pi
t=1
equation = PDE(net, a, l,t)
train = Train(net, equation, BATCH_SIZE=2**8)
train.train(epoch=1500, lr=0.0001)
torch.save(net, 'test.pkl')
errors = train.get_errors()

Epoch 49 - lr 0.0001 -  loss: 1.0489021706581116
Epoch 99 - lr 0.0001 -  loss: 0.9181552374362946
Epoch 149 - lr 0.0001 -  loss: 0.7375347304344178
Epoch 199 - lr 0.0001 -  loss: 0.6750632786750793
Epoch 249 - lr 0.0001 -  loss: 0.49285370349884033
Epoch 299 - lr 0.0001 -  loss: 0.21372802823781967
Epoch 349 - lr 0.0001 -  loss: 0.13732985004782677
Epoch 399 - lr 0.0001 -  loss: 0.10072590097784996
Epoch 449 - lr 0.0001 -  loss: 0.06477287232875824
Epoch 499 - lr 0.0001 -  loss: 0.053275579288601875
Epoch 549 - lr 0.0001 -  loss: 0.0359943887591362
Epoch 599 - lr 0.0001 -  loss: 0.03590043190866709
Epoch 649 - lr 0.0001 -  loss: 0.021922803670167922
Epoch 699 - lr 0.0001 -  loss: 0.02135512441396713
Epoch 749 - lr 0.0001 -  loss: 0.016542741991579533
Epoch 799 - lr 0.0001 -  loss: 0.015779891069978475
Epoch 849 - lr 0.0001 -  loss: 0.015574104096740484
Epoch 899 - lr 0.0001 -  loss: 0.012272395845502614
Epoch 949 - lr 0.0001 -  loss: 0.012772510964423419
Epoch 999 - lr 0.0001 -  loss: 

In [21]:
fig = plt.figure()
plt.plot(np.log(errors), '-b', label='Errors')
plt.title('Training Loss', fontsize=10)
plt.savefig('error.jpg')
plt.close(fig)

C:\Users\cm\anaconda3\envs\pytorch\lib\site-packages\matplotlib\animation.py:879: UserWarning: Animation was deleted without rendering anything. This is most likely not intended. To prevent deletion, assign the Animation to a variable, e.g. `anim`, that exists until you output the Animation using `plt.show()` or `anim.save()`.
  warnings.warn(


In [13]:
import PyQt5
%matplotlib qt5

In [20]:
import matplotlib.pyplot as plt
from matplotlib import animation
import numpy as np
net=torch.load('test.pkl')
fig, ax = plt.subplots()
err=[]
# 定义存储数据的列表
xdata = []
ydata = []

# 接收line2D对象
line, = plt.plot(xdata, ydata, 'b')
xdata=[i/100 for i in range(0,314)]
# 定义更新函数
def update(frames):
    x_1 = torch.tensor([[i/100] for i in range(0,314)])
    x_11 = torch.cat((torch.full([314, 1],frames), x_1), dim=1)
    ysolve1=net(x_11)
    ysolve=ysolve1.detach().numpy()
    ydata=[ysolve]
    line.set_data(xdata, ydata)
    return line


def init_figure():
    ax.set_xlim(0, 3.14)
    ax.set_ylim(-2,2)
    ax.set_xlabel('x')
    ax.set_ylabel('u')


# 调用生成动画的函数生成动图
ani = animation.FuncAnimation(
    fig=fig,
    func=update,
    frames=np.linspace(0, 1, 100),    # [1, 2, 3]
    init_func=init_figure,
    interval=20,   #  每隔多少时间生成一帧图像，单位是ms
    repeat=True,   # 设置不重复，但是还是重复较好
)

#plt.show()   # 如果要保存视频和gif就不要show()
#ani.save('shuli.gif', writer='pillow')
ani.save('1.mp4', writer='ffmpeg')  # 注意，pillow现在似乎不能报错为mp4格式了，可以使用ffmpeg